# ABM

## Imports / Data Conversion

In [176]:
import time, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import random
from tqdm import tqdm, trange
from time import sleep
import pickle
import csv

data_path = '../' #set to wherever the data files are, will be used on every input

In [131]:
# def kmtoNaut(km):
#     return km / 1.852

In [132]:
"""
Import of preprocessed Network Data
"""
origin = pd.read_csv((data_path + 'origin_ports.csv'))
data = pd.read_csv((data_path + 'clean_distances.csv')) # i keep the old name
distances = data[["prev_port", "next_port", "distance"]] 
# distances = distances_df[["prev_port", "next_port", "distance"]]
# distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes
# ports = pd.read_csv((data_path +'ports.csv'))

In [133]:
'''
Route Blockages Import
'''
route_blockage_dov = pd.read_csv((data_path + 'route_blockages_dov.csv'))
route_blockage_gib = pd.read_csv((data_path + 'route_blockages_gib.csv'))
route_blockage_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))
route_blockage_mal = pd.read_csv((data_path + 'route_blockages_mal.csv'))
route_blockage_pan = pd.read_csv((data_path + 'route_blockages_pan.csv'))
route_blockage_suez = pd.read_csv((data_path + 'route_blockages_suez.csv'))
route_blockage_total = pd.read_csv((data_path + 'route_blockages_total.csv'))

In [134]:
'''
Pruning Files List is uses to pass all route blockage files at once to the Model
'''
pruning_files = [route_blockage_dov, route_blockage_gib, route_blockage_horm, route_blockage_mal, route_blockage_pan, route_blockage_suez, route_blockage_total]

In [135]:
pruning_schedule={10:'Gibraltar', 15:"Open"}

## Model

We use a Mesa ABM on a Network Grid that is based on the route / ports data we have obtained. We are using a 

In [188]:
class ShippingNetwork(Model):
    def __init__(self, distances, major_ports, pruning_files, pruning_schedule ,S=1000,  s=20, f = 0, x = 0.1):
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.Ships = []
        self.pruning_files = pruning_files
        self.s = s
        self.f = f
        self.x = x
        self.pruning_schedule = pruning_schedule
        self.stp_cnt = 0

        '''
        ensure use of correct network graph, correct nomenclature 
        '''
        #Build Network without closures
        self.G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())    
        #Define Mesa Grid as the just created Network to allow for shipping only in routes
        self.grid = NetworkGrid(self.G) 

        '''
        Build alternate Networks
        '''
        #Build alternate Networks (with closures in place)
        self.G_Dov = self.Cut_Graph(self.G, self.pruning_files[0])
        self.G_Gib = self.Cut_Graph(self.G, self.pruning_files[1])
        self.G_Horm = self.Cut_Graph(self.G, self.pruning_files[2])
        self.G_Mal = self.Cut_Graph(self.G, self.pruning_files[3])
        self.G_Pan = self.Cut_Graph(self.G, self.pruning_files[4])
        self.G_Suez = self.Cut_Graph(self.G, self.pruning_files[5])
        self.G_Total = self.Cut_Graph(self.G, self.pruning_files[6])
  


        #create agents 
        Ships = []
       
        for i in range(self.num_ships):
        
            a = Ship(i+1, self, self.G, self.major_ports,  self.s, self.f, self.x)
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
        
            #place agent on origin node
            self.grid.place_agent(a, a.start)



        self.datacollector = DataCollector(
            model_reporters={"Graph":"blockage"},
            agent_reporters={"Type": "ship_class","Foresight": "foresight", "State":"state", "Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng", "Destination not reachable" : "not_reachable", "Sucess": "sucess", "Stuck":"stuck"})


        '''
        Function to cut graphs (used in the setup phase)
        '''
    def Cut_Graph(self, G, route_blockages):
        return_G = G.copy()  #CRUCIAL TO INCLUDE COPY STATEMENT
        for index in range(len(route_blockages)):
            try:
                return_G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                pass
        return return_G


    '''
    Method allows for change of network (copies specified pre-built )
    '''
        #create ability to remove edges mid-model
    def network_change(self, blockage):
        if blockage == "Dover":
            G_new = self.G_Dov
        elif blockage == "Gibraltar":
            G_new = self.G_Gib
        elif blockage == "Hormuz":
            G_new = self.G_Horm
        elif blockage == "Malacca":
            G_new = self.G_Mal
        elif blockage == "Panama":
            G_new = self.G_Pan
        elif blockage == "Suez":
            G_new = self.G_Suez
        elif blockage == "Total":
            G_new = self.G_Total
        elif blockage == "Open":
            G_new = self.G

        for a in self.Ships:
                a.G = G_new

    def network_check(self):
        try:
            self.network_change(self.pruning_schedule[self.stp_cnt])
        except:
            pass


    def step(self):
        self.network_check()
        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)
        self.stp_cnt += 1




In [189]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, major_ports, s, f, x):
        super().__init__(unique_id, model)
        
        self.major_ports = major_ports
        self.G = G
        self.s = s
        self.f = f
        self.x = x
        self.factor = 1+self.x

        self.ship_class = np.random.choice(["Ultralarge","Normal", "Small"], 1, p=[0.5, 0.25, 0.25])
        self.start_port = self.origin()
        
        self.destination = self.dest() #sample a destination
        #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
        self.ports =  [*self.start_port, *self.destination]
        
        self.foresight = np.random.poisson(self.f)
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = self.s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)

        self.not_reachable = 0 #counter for needing to abandon a route 

        self.new_dest_failed = 0 #counter for ships not able to reach any of the ports

        self.init_route, self.init_dist  = self.routing() #We keep a copy of the entire itinerary / distance traveled
        self.start = self.start_port[0]
        self.current_route, self.current_dist = self.init_route.copy(), self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.start_speed = self.speed
        self.position = self.current_route[0]
        self.next_position = self.current_route[1]
        self.target  = (self.init_dist // self.start_speed) * self.factor #target to reach all destinations
        self.itinerary = [self.position]
        self.distance_traveled = 0
        self.unique_id = unique_id
        
        self.step_size = self.ident_distance()
        self.route_chng = 0
        self.complete_route = 0
        self.steps = 0
        self.sucess = 0
        self.stuck = 0

    def origin(self):
        """
        Sample origin based on ship type.
        """ 
        
        if self.ship_class == "Ultralarge":
            start_port = np.random.choice(self.major_ports["Ref"],  p=self.major_ports["PROB"])

        elif self.ship_class == "Normal":
            start_port = np.random.choice(self.major_ports["Ref"],  p=self.major_ports["PROB"])

        else:
            r = random.sample(self.G.nodes, k=1)[0]
            if not nx.is_isolate(self.G,r):
                start_port = float(r)
            else:
                return origin()
            
        return [start_port]
        

    def dest(self):
        """
        Sample destinations
        """
        if self.ship_class == "Ultralarge": #large ships only visit large ports

            #we try to mix in top 10 ports with the rest

            p1 = self.major_ports["PROB"][:10].copy()
            p1 /= p1.sum()
            p2 = self.major_ports["PROB"][10:].copy()
            p2 /= p2.sum()
            k = np.random.randint(1, high = 3)
            end = np.random.choice(self.major_ports["Ref"][:10] , size=k,  p=p1).tolist() + np.random.choice(self.major_ports["Ref"][10:], size=k,  p=p2).tolist()
                        
            
        elif self.ship_class == "Normal":
            k = np.random.randint(1, high = 4)
            end = np.random.choice(self.major_ports["Ref"], replace=False, size=k,  p=self.major_ports["PROB"]).tolist()+ [float(i) for i in random.sample(self.G.nodes, k=k)]

        else:
            k = np.random.randint(1, high =6)
            end = [float(i) for i in random.sample(self.G.nodes, k=k)]

        return end

    def routing(self):
        """
        A greedy version of Travelling Salesman algorithm.
        Takes in a list of ports, with the first port being the origin.
        It loops to find the closest port. Returns a list of ports to visit (an itinerary) and the overall distance.
        """
        ports = self.ports.copy()
        overall_distance = list()
        itinerary = list() 
        not_reached = 0
        
        try:
            for j in range(len(ports)):
                distance = dict()
                try:
                    for i in range(1,len(ports)): #look for the closest port
                        distance[ports[i]] = nx.shortest_path_length(self.G, ports[0] , ports[i], weight='distance')
                    next_stop = min(distance, key=distance.get)
                    itinerary.append(nx.shortest_path(self.G, ports[0], next_stop, weight = 'distance')) #add the route to the closest port to the itinerary
                    overall_distance.append(distance.get(next_stop)) #add distance to the closest port
                    ports.pop(0)
                    ind = ports.index(next_stop)
                    ports.pop(ind)
                    ports.insert(0, next_stop)

                except nx.NetworkXNoPath:
                    self.not_reachable += 1 #global counter
                    not_reached += 1 #loccal counter

        except ValueError: #handle list end
            pass

        if not_reached == len(ports): #if no routes possible routes found, reassign destination
            self.new_dest_failed += 1
            self.destination = self.dest()
            self.ports =  [*self.start_port, *self.destination]
            return self.routing()

            if self.new_dest_failed > 1: #if the problem persists, the ship is stuck
                self.stuck += 1
                

            else:
                return self.routing() #recursion brrrr
        else:   
            flat_route = []
            for sublist in itinerary: #flatten the itinerary
                for port in sublist:
                    flat_route.append(port)

            
            travel_distance = sum(overall_distance)
            return flat_route, travel_distance

    def move(self):

        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.pop(0) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route


    def ident_distance(self): #look up the distance of the current step
        try:
            return self.G.get_edge_data(self.position, self.next_position, default=0)['distance']
        except:
            return 0
    
    def new_destinations(self):
        
        self.complete_route += 1
        self.destination = self.dest()
        self.ports =  [self.position, *self.destination]
        self.init_route, self.init_dist = self.routing()
        self.current_route, self.current_dist = self.init_route.copy(), self.init_dist
        self.target  = (self.init_dist // self.start_speed)  * self.factor
        self.state = 3 #WAIT AT PORT 

 
    def step(self):
        self.state = self.state - 1 #'move' ships by one day progress
        if self.stuck >=1:
            pass
        else:
            if round(self.state,3) <= 0: #ships that are en-route to the node they are going to next do not move / perform other activities
                self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
            
                #add the current position to itinerary
                if self.position != self.current_route[-1]: #if current stop is not the final stop
                    self.ports =  [self.position, *self.destination]
                    new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                    
                    if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                        # print("default case")
                        self.move()
                        self.itinerary.append(self.position)
                        self.steps +=1
            
                # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                    elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                        

                        if self.foresight <= (self.current_dist // self.speed): #check how many steps are you from your final destination. If you are far away, do nothing and remain on course
                            # if not has_path(self.G, self.position, self.next_position): 
                            self.current_route = new_route
                            self.current_dist = new_distance
                            self.route_chng += 1
                            self.move()
                            self.itinerary.append(self.position)
                            self.steps +=1
                        else:
                            self.move()
                            self.itinerary.append(self.position)
                            self.steps +=1
                    
                    
                    else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                        

                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                        self.move()
                        self.itinerary.append(self.position)
                        self.steps +=1
                
                else: #if ship is arrived at final position, get a new route, and start back
                    if self.steps <= self.target: #if the ship manages the reach all the destinations in time, it is "sucessful"
                        self.sucess += 1
                        self.new_destinations()
                        self.itinerary.append(self.position)
                        self.steps = 0 #clear the counter
                    else:
                        self.new_destinations()
                        self.itinerary.append(self.position)
                        self.steps = 0 #clear the counter

## Model Instances

In [191]:
model = ShippingNetwork(distances, origin, pruning_files, pruning_schedule, 1)

steps = 20
for i in trange(steps):
    model.step()


agent_state = model.datacollector.get_agent_vars_dataframe()

# , "x": np.arange(-0.75, 1.25, 0.25)

100%|██████████| 20/20 [00:09<00:00,  2.09it/s]


In [158]:
from mesa.batchrunner import BatchRunner

fixed_params = {"distances": distances, "major_ports":origin, "pruning_files": pruning_files, "pruning_schedule": pruning_schedule, "S": 5}
variable_params = {"f": range(0, 20, 5), "x": np.arange(-0.75, 1.25, 0.25)}

batch_run = BatchRunner(ShippingNetwork,
                        variable_params,
                        fixed_params,
                        iterations=1,
                        max_steps=20,
                       
                        )
batch_run.run_all()

0it [00:00, ?it/s]0
1
2
3
4
5
6
7
8
9
Gibraltar 10
11
12
13
14
Open 15
16
17
18
19
1it [00:55, 56.00s/it]


RecursionError: maximum recursion depth exceeded

In [120]:
data_collector_agents = batch_run.get_collector_agents()

In [178]:
keys = data_collector_agents.keys()

In [181]:
list(keys)

[(0, 0.0, 0),
 (0, 0.25, 1),
 (0, 0.5, 2),
 (0, 0.75, 3),
 (0, 1.0, 4),
 (5, 0.0, 5),
 (5, 0.25, 6),
 (5, 0.5, 7),
 (5, 0.75, 8),
 (5, 1.0, 9),
 (10, 0.0, 10),
 (10, 0.25, 11),
 (10, 0.5, 12),
 (10, 0.75, 13),
 (10, 1.0, 14),
 (15, 0.0, 15),
 (15, 0.25, 16),
 (15, 0.5, 17),
 (15, 0.75, 18),
 (15, 1.0, 19)]

In [182]:
"""
write output to file
"""

with open('../test.pickle', 'wb') as handle:
    pickle.dump(data_collector_agents, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../keys.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for a, b, c in list(keys):
       writer.writerow([a, b, c])

In [184]:
"""
Load files for analysis
"""

with open('../test.pickle', 'rb') as handle:
    b = pickle.load(handle)

with open('../keys.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = list(reader)

[['0', '0.0', '0'],
 ['0', '0.25', '1'],
 ['0', '0.5', '2'],
 ['0', '0.75', '3'],
 ['0', '1.0', '4'],
 ['5', '0.0', '5'],
 ['5', '0.25', '6'],
 ['5', '0.5', '7'],
 ['5', '0.75', '8'],
 ['5', '1.0', '9'],
 ['10', '0.0', '10'],
 ['10', '0.25', '11'],
 ['10', '0.5', '12'],
 ['10', '0.75', '13'],
 ['10', '1.0', '14'],
 ['15', '0.0', '15'],
 ['15', '0.25', '16'],
 ['15', '0.5', '17'],
 ['15', '0.75', '18'],
 ['15', '1.0', '19']]